In [1]:
# favorite example

# cam2, start_index = 20, end_index = 23

# point №40
# it corresponds to instance 53 (image 20)
# it corresponds to instance 45 (image 21)
# it corresponds to instance 53 (image 22)

In [2]:
from pathlib import Path

from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset/'
sequence = '00'
sam_labels_path = Path.cwd().joinpath('pipeline', 'vfm-labels', 'sam', sequence)

kitti = KittiDataset(dataset_path, sequence, sam_labels_path)

In [3]:
from mapping.simple_mapping import SimpleMapping

sm = SimpleMapping(kitti)
coo_matrix, pcds = sm.get_combined_labeled_point_clouds('cam2', 20, 23)

INFO:simple_mapping.SimpleMapping:Image 20 processed
INFO:simple_mapping.SimpleMapping:Image 21 processed
INFO:simple_mapping.SimpleMapping:Image 22 processed


In [4]:
from mapping.sparse_matrix_utils import construct_coo_matrix_for_multiple_views

coo_result = construct_coo_matrix_for_multiple_views(coo_matrix, 20)

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

In [6]:
enc.fit((coo_result.T).toarray())

OneHotEncoder(handle_unknown='ignore')

In [7]:
enc.categories_

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 37., 38., 39., 40.,
        42., 43., 44., 45., 46., 48., 49., 50., 51., 53., 54., 56., 57.,
        58., 59., 62., 63., 64., 67., 68., 69., 71., 73., 74., 75., 76.,
        77., 78., 79., 80.]),
 array([ 0.,  2.,  3.,  4.,  5.,  6.,  7., 10., 11., 12., 13., 14., 15.,
        16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
        29., 30., 31., 32., 33., 34., 35., 36., 37., 39., 40., 41., 42.,
        43., 44., 45., 47., 49., 50., 51., 52., 54., 55., 56., 58., 59.,
        60., 62., 64., 65., 67., 69., 71., 72., 73., 74.]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 4

In [8]:
enc_coo_result = enc.transform(coo_result.T.toarray()).toarray()

In [9]:
inverse_enc_coo_result = enc.inverse_transform(enc_coo_result)

k = 0
for itr in range(len(inverse_enc_coo_result)):
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
        
print("Number of points with non-zero instances: {}".format(len(inverse_enc_coo_result) - k))

Number of points with non-zero instances: 92411


In [10]:
summary_pcd = pcds[0] + pcds[1] + pcds[2]

In [11]:
inverse_enc_coo_result = enc.inverse_transform(enc_coo_result)
points = []
map_old_new_point_ind = {}

k = 0
for itr in range(len(inverse_enc_coo_result)):
    if (inverse_enc_coo_result[itr] == [0.0, 0.0, 0.0]).all():
        k = k + 1
    else:
        if k < 20:
            print(itr)
        points.append(summary_pcd.points[itr])
        dict1 = {itr:(len(points) - 1)}
        map_old_new_point_ind.update(dict1)

0
1
2
3
4
5
20
21
22
23
24
25
26
27
28
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


In [12]:
enc_coo_non_zero_instances = []
for itr in range(len(enc_coo_result)):
    if itr in map_old_new_point_ind.keys():
        enc_coo_non_zero_instances.append(enc_coo_result[itr])

In [13]:
inverse_enc_coo_non_zero_instances = enc.inverse_transform(enc_coo_non_zero_instances)
print(inverse_enc_coo_non_zero_instances[990])

[ 0. 25. 15.]


In [14]:
import open3d as o3d

pcd_baby = o3d.geometry.PointCloud()
pcd_baby.points = o3d.utility.Vector3dVector(points)

In [15]:
downpcd = pcd_baby.voxel_down_sample(voxel_size=0.05)
# o3d.visualization.draw_geometries([downpcd])

In [16]:
# sresss = []
# mapsresss = {}
# for itr in range(len(s)):
#     if itr in map.keys():
#         sresss.append(s[itr])
#         dict12 = {itr:len(sresss) - 1}
#         mapsresss.update(dict12)

In [17]:
# i = enc.inverse_transform(enc_coo_result)
# 
# start = 20
# end = 46
# for point in range(start, end) :
#     print("{} : {}".format(point, i[point]))

In [18]:
from sklearn.metrics.pairwise import pairwise_distances
distances = pairwise_distances(enc_coo_non_zero_instances[:1000])

In [19]:
distances.shape

(1000, 1000)

In [20]:
# import numpy as np
# W = np.array(distances, copy=True)

In [21]:
from scipy import sparse
t = sparse.identity(distances.shape[0])

In [22]:
t

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1000 stored elements (1 diagonals) in DIAgonal format>

In [23]:
# for i in range(distances.shape[0]):
#     distances[i, i] = distances[i, i] + 1.0

In [24]:
sum = distances + t

In [25]:
sum

matrix([[1., 0., 0., ..., 2., 2., 2.],
        [0., 1., 0., ..., 2., 2., 2.],
        [0., 0., 1., ..., 2., 2., 2.],
        ...,
        [2., 2., 2., ..., 1., 0., 0.],
        [2., 2., 2., ..., 0., 1., 0.],
        [2., 2., 2., ..., 0., 0., 1.]])

In [26]:
len(pcd_baby.points[:50000])

50000

In [27]:
arr = pcd_baby.points[:10000]
print(arr[1, 3])

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.utility.Vector3dVector, s: slice) -> open3d.cpu.pybind.utility.Vector3dVector
    2. (self: open3d.cpu.pybind.utility.Vector3dVector, arg0: int) -> numpy.ndarray[numpy.float64[3, 1]]

Invoked with: std::vector<Eigen::Vector3d> with 10000 elements.
Use numpy.asarray() to access data., (1, 3)

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.

In [52]:
from awesome_normalized_cut import normalized_cut
import numpy as np
labels = np.arange(1000)

clusters = normalized_cut(distances, labels, 1.10000000000005)
# clusters = normalized_cut(distances, pcd_baby.points[:1000], 10)

1
(1000, 1000)
2
3
4
5
6
7
8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0007539796182967
mask=[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  T

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 19 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 27 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


8
9
10
01
02
03
04
05
07
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
011
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
08
09
001
002
003
004
010
012
11
1.0017243392583142
mask=[ True  True False False False False False  True False False  True  True
  True False False False  True False False False  True False False False
 False False False False  True False False  True False  True False False
 False False  True False False  True False  True  True  True  True False
  True  True False False False False False False False  True False  True
 False False False False  True False  True False  True False False False
 False  True  True False False  True False False False  True  True False
  True  True  True False False False False False  True False  True False
  True False False  True False  True False  True  True  True False  True
 False False Fa

/Users/sofiavivdich/projects/lidar-labelling/venv/lib/python3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 12 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


In [54]:
clusters

[array([272, 692, 878]),
 array([ 61, 563, 736, 891, 903]),
 array([127, 252, 764, 941]),
 array([ 36, 185, 423, 668, 893]),
 array([ 75, 110, 140, 335, 678]),
 array([433, 583, 687]),
 array([477]),
 array([323, 956]),
 array([ 60, 224, 503, 683, 823]),
 array([ 94, 234, 499]),
 array([ 22, 177, 375]),
 array([277, 539, 721]),
 array([373, 554]),
 array([397]),
 array([457]),
 array([ 44, 439, 633, 693, 901]),
 array([321, 528, 826]),
 array([871]),
 array([ 27,  49, 237, 567, 857]),
 array([429, 611]),
 array([182, 379, 415]),
 array([197, 362, 628, 756, 869]),
 array([384]),
 array([128]),
 array([124, 691, 763]),
 array([419, 607, 629]),
 array([ 82, 133, 656]),
 array([441, 625]),
 array([204, 349, 380, 483]),
 array([939]),
 array([ 21, 157, 209, 403, 815]),
 array([14]),
 array([100, 636]),
 array([ 99, 132, 690, 970]),
 array([390, 626, 972]),
 array([299, 463, 806]),
 array([313, 920]),
 array([351]),
 array([597, 684, 750, 935]),
 array([428, 556]),
 array([307, 367, 377, 879

In [55]:
inverse_enc_coo_non_zero_instances[61]

array([48., 41., 50.])

In [56]:
inverse_enc_coo_non_zero_instances[563]

array([ 0., 13., 30.])

In [43]:
clusters[0][9999]

array([9.29407103, 0.40789549, 0.18582554])

In [12]:
# 1) выбросить ненужные точки. создать новое облако и мапу старое - новое
# 2) voxel_down_sample применить к маленькому облаку
# применить к нему граф кат, раскрасить по кластерам ИСХОДНОЕ маленькое облако с новыми инстансами
# 3) kd деревья почитать чтобы найти соседей